In [63]:
# Widen width of notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [64]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

import numpy as np 
import pandas as pd
import string
import re 

from nltk.corpus import stopwords
import nltk
# nltk.download('stopwords')
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import WordPunctTokenizer
from gensim.parsing.preprocessing import remove_stopwords,strip_numeric,strip_non_alphanum,strip_punctuation,strip_multiple_whitespaces

from scipy import sparse
from scipy.spatial.distance import cdist

import warnings; warnings.simplefilter('ignore')

In [65]:
nv = pd.read_csv('filted_nv.csv')

In [66]:
# Further remove noise
df_nv = nv[nv['categories'].str.contains('Refinishing') == False]
df_nv = df_nv[df_nv['categories'].str.contains('Hotels') == False]
df_nv = df_nv[df_nv['categories'].str.contains('Event Planning & Services') == False]
df_nv = df_nv[df_nv['categories'].str.contains('Shopping') == False]

df_review_user =  df_nv.loc[:][['user_id','business_id','categories','stars_r','stars_b','text']]

# pd.set_option('display.max_colwidth', None)
# df_review_user[500:550]

In [67]:
# Remove review where text = nan
df_review_user = df_review_user[~df_review_user['text'].isnull()]
df_review_user[df_review_user['text'].isnull() == True]

,user_id,business_id,categories,stars_r,stars_b,text


In [68]:
df_review_user.shape

(1249176, 6)

In [69]:
# # Initialize the word cloud
# wc = WordCloud(width = 500, height = 500, min_font_size = 10, background_color ='white')
# cate = df_nv['text'].str.cat(sep=' ')
# cate_wc = wc.generate(cate)
    
# # plot
# plt.figure(figsize = (5, 5), facecolor = None) 
# plt.imshow(cate_wc) 
# plt.axis("off") 
# plt.title("Word Cloud for Reviews")
# plt.tight_layout(pad = 0) 
# plt.show()

In [70]:
contractions = {
"ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because",
"could've": "could have","couldn't": "could not","couldn't've": "could not have","didn't": "did not",
"doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have",
"hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have",
"he'll": "he will","he'll've": "he will have","he's": "he is","how'd": "how did","how'd'y": "how do you",
"how'll": "how will","how's": "how is","i'd": "I would","i'd've": "I would have","i'll": "I will",
"i'm": "I am","i've": "I have","isn't": "is not","it'd": "it would","it'd've": "it would have",
"it'll": "it will","it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have",
"mightn't": "might not","mightn't've": "might not have","must've": "must have","mustn't": "must not",
"mustn't've": "must not have","needn't": "need not","needn't've": "need not have","o'clock": "of the clock",
"oughtn't": "ought not","oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
"shan't've": "shall not have","she'd've": "she would have","she'll": "she will","she's": "she is",
"should've": "should have","shouldn't": "should not","shouldn't've": "should not have","that's": "that is",
"there'd've": "there would have","there's": "there is","they'd've": "they would have","they'll": "will",
"they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd've": "we would have",
"we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have","weren't": "were not",
"what're": "what are","what's": "what is","what've": "what have","when've": "when have","where'd": "where did",
"where's": "where is","where've": "where have","who'll": "who will","who's": "who is","who've": "who have",
"why've": "why have","will've": "will have","won't": "will not","won't've": "will not have","would've": "would have",
"wouldn't": "would not","wouldn't've": "would not have","y'all": "you all","y'all'd": "you all would",
"y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would",
"you'd've": "you would have","you'll": "you will","you're": "you are","you've": "you have"
}

In [71]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

def clean_text(text):
    
    try:
        text = text.lower()
    
        # Replace contractions
        for word in text.split():
            if word.lower() in contractions:
                text = text.replace(word, contractions[word.lower()])
        
        for word in text.split():
            if isEnglish(word) == False:
                text = text.replace(word," ")
        
#         text = re.sub("[^a-zA-Z0-9]+"," ",text)
        text = strip_multiple_whitespaces(text)
        text = strip_numeric(text)
        text = text.translate(str.maketrans(' ',' ','!"#$%&\'()*+,./:-;<=>?@[\\]^_`{|}~'))
        
        if isEnglish(text) == False:
            text = ""
    except:
        a = 1
    return text.strip()

In [72]:
# text = "What's the a not the best way to ensure this?"
# clean_text(text)
df_review_user['text_clean'] = df_review_user['text'].apply(clean_text)

In [73]:
# Remove other languages
df_review_user['text_clean'].replace('', np.nan, inplace=True)

# check null 
df_review_user = df_review_user[~df_review_user['text_clean'].isnull()]
df_review_user[df_review_user['text_clean'].isnull() == True]

,user_id,business_id,categories,stars_r,stars_b,text,text_clean


## ngram_range (1,3)

In [74]:
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
all_stopwords.remove('nor')

# vectorizer_reviews = CountVectorizer(stop_words = all_stopwords,ngram_range = (1,3))
vectorizer_reviews = CountVectorizer(min_df = .001,stop_words = all_stopwords,ngram_range = (1,3))

In [75]:
vectorized_reviews = vectorizer_reviews.fit_transform(df_review_user['text_clean'])

In [76]:
' | '.join(vectorizer_reviews.get_feature_names()[-100:])

'would great | would highly | would highly recommend | would like | would liked | would love | would make | would much | would never | would nice | would not | would not come | would not even | would not go | would not order | would not recommend | would order | would ordered | would pay | would preferred | would probably | would rate | would rather | would recommend | would recommend anyone | would recommend place | would return | would say | would still | would suggest | would take | would think | would try | would want | wow | wow place | wowed | wrap | wrapped | wraps | write | write home | write review | writing | writing review | written | wrong | wrong food | wrong not | wrong order | wrote | wtf | wynn | ya | yard | yard house | yay | yea | yeah | year | year ago | year old | years | years ago | years never | yell | yelling | yellow | yellow curry | yellowtail | yelp | yelp review | yelp reviews | yelpers | yep | yes | yesterday | yet | yet not | yo | yogurt | yolk | york | yor

In [77]:
# pd.set_option('display.max_colwidth', None)
# df_review_user[df_review_user['text_clean'].str.contains('zz') == True]
# # # # df_review_user[df_review_user['text_clean'] == ""]

In [78]:
def clean_cat_text(text):
    text = text.replace("Restaurants"," ")       
    return text

df_business_cat = df_review_user.drop_duplicates(subset=['business_id'], keep='last')

In [79]:
df_business_cat['new_categories'] = df_business_cat['categories'].apply(clean_cat_text)

In [80]:
vectorizer_categories = CountVectorizer(min_df = 1, max_df = 1., tokenizer = lambda x: x.split(', '))
vectorized_categories = vectorizer_categories.fit_transform(df_business_cat['new_categories'])

In [81]:
' | '.join(vectorizer_categories.get_feature_names()[:100]) # only the first 100

'  | acai bowls | active life | acupuncture | adult entertainment | afghan | african | air duct cleaning | aircraft repairs | airsoft | amateur sports teams | american (new) | american (traditional) | amusement parks | animal assisted therapy | animal physical therapy | animal shelters | arabian | arcades | argentine | armenian | arts & entertainment | asian fusion | austrian | auto customization | auto detailing | auto glass services | auto insurance | auto parts & supplies | auto repair | automotive | bagels | bakeries | bangladeshi | bankruptcy law | banks & credit unions | bar crawl | barbeque | bars | beach bars | beauty & spas | beer | beer bar | beer garden | beer gardens | belgian | bingo halls | bistros | body shops | bowling | brasseries | brazilian | brazilian jiu-jitsu | breakfast & brunch | breweries | brewpubs | british | bubble tea | buffets | burgers | business consulting | butcher | cabaret | cafes | cafeteria | cajun/creole | candy stores | cantonese | car wash | car 

In [82]:
businessxreview = sparse.csr_matrix(pd.get_dummies(df_review_user['business_id']).values)

In [83]:
print('restuarants x categories: \t', vectorized_categories.shape) 
print('restuarants x reviews: \t\t' , businessxreview.shape) 
print('reviews x words: \t\t', vectorized_reviews.shape)

restuarants x categories: 	 (4906, 365)
restuarants x reviews: 		 (1248718, 4906)
reviews x words: 		 (1248718, 7540)


In [84]:
user_selection = '4JNXUYY8wbaaDmk3BPzlWw'

In [85]:
business_reviews = df_review_user.loc[df_review_user['business_id'] == user_selection, 'text_clean']
business_categories = df_review_user.loc[df_review_user['business_id'] == user_selection, 'categories']

In [86]:
# find most similar reviews
reviewdist = cdist(vectorizer_reviews.transform(business_reviews).todense().mean(axis=0), 
              vectorized_reviews.T.dot(businessxreview).T.todense(), 
               metric='correlation')

# find most similar categories
catdist = cdist(vectorizer_categories.transform(business_categories).todense().mean(axis=0), 
              vectorized_categories.todense(), 
               metric='correlation')

dists_cat_review = np.vstack([reviewdist.ravel(), catdist.ravel()]).T
dists = dists_cat_review.mean(axis=1)

In [87]:
closest = dists.argsort().ravel()

biz_index = closest[1:11]
biz_list = df_business_cat.iloc[biz_index]['business_id']

In [88]:
# Retrieve user selection
df_nv.loc[df_nv['business_id']== '4JNXUYY8wbaaDmk3BPzlWw'][['business_id','categories','name_b','stars_b']].drop_duplicates(subset=['business_id'], keep='first')

,business_id,categories,name_b,stars_b
7,4JNXUYY8wbaaDmk3BPzlWw,"Restaurants, French, Steakhouses, Food, Breakfast & Brunch",Mon Ami Gabi,4.0


In [89]:
# Retrieve Top 10 similarity - Check
pd.set_option('display.max_colwidth', None)
df_nv.loc[df_nv['business_id'].isin(biz_list)][['business_id','categories','name_b','stars_b']].drop_duplicates(subset=['business_id'], keep='first').head(20)

,business_id,categories,name_b,stars_b
226,IWN2heYitkg-D4UdqfxcMA,"Nightlife, Bars, Restaurants, Cocktail Bars, Breakfast & Brunch, Desserts, Food, Steakhouses, American (New)",HEXX kitchen + bar,4.0
443,TXS-9SUkjpZFz29XOB1SPg,"Coffee & Tea, Restaurants, Cafes, Food, Soul Food, Breakfast & Brunch",M&M Soul Food Cafe,2.5
625,DY1_vls0_VQnTZ7CJ3AeFQ,"Buffets, Restaurants, Food, Breakfast & Brunch",Feast Buffet,3.0
742,KalAJyO0Zpg3K1wVwYXBHA,"Food, Beer, Wine & Spirits, Wine Bars, Nightlife, French, Bars, Restaurants, Specialty Food, Breakfast & Brunch",Marché Bacchus,3.5
1469,PkqN5st3zZ_Xce_5QlBtNQ,"Breakfast & Brunch, Restaurants, Coffee & Tea, Hawaiian, Food",Cinnamon's,4.0
1940,BcW7Z9lPmOB_8eS2lEuOqQ,"Japanese, Steakhouses, Sushi Bars, Restaurants, Food",Musashi Japanese Steakhouse,4.0
2517,XZ7eQLjyGXf6y69UT4SGhg,"Cafes, Food, Creperies, Bakeries, French, Breakfast & Brunch, Restaurants",Cafe Breizh,4.5
4917,jlpseJr1CS8aCMieENkQ8g,"Breakfast & Brunch, Delis, Bagels, Food, Restaurants",Einstein Bros,3.0
17148,T5nTtSCSRoEYMsZPhF61wg,"Coffee & Tea, Juice Bars & Smoothies, Restaurants, Food, Breakfast & Brunch",Waffles Cafe,4.0
41501,UGy0QULAPPRcOUJQSiRmGQ,"Restaurants, Sandwiches, Juice Bars & Smoothies, Food, Breakfast & Brunch",Tropical Smoothie Cafe,3.0


In [90]:
business_to_predict = [x for x in closest if x <= 0.55]